In [2]:
import pandas as pd

# Store filepath in a variable
crime_data = "crime-data.csv"

# Read our Data file with the pandas library
crime_data = pd.read_csv(crime_data, low_memory=False)

#crime_data_df = pd.DataFrame(pd.read_csv("crime-data.csv"), low_memory=False)
crime_data_df = pd.DataFrame(crime_data)

#crime_data_df.dtypes
crime_data_df.head()


,INC NUMBER,OCCURRED ON,OCCURRED TO,UCR CRIME CATEGORY,100 BLOCK ADDR,ZIP,PREMISE TYPE
0,201500000000000,11/1/15 12:00 AM,11/1/15 5:00 AM,MOTOR VEHICLE THEFT,102XX W MEDLOCK AVE,85307.0,SINGLE FAMILY HOUSE
1,201500000000000,11/1/15 12:00 AM,11/1/15 11:50 AM,MOTOR VEHICLE THEFT,69XX W WOOD ST,85043.0,SINGLE FAMILY HOUSE
2,201600000000000,11/1/15 12:00 AM,1/9/16 12:00 AM,MOTOR VEHICLE THEFT,N 43RD AVE & W CACTUS RD,85029.0,SINGLE FAMILY HOUSE
3,201500000000000,11/1/15 12:00 AM,11/11/15 9:30 AM,LARCENY-THEFT,14XX E HIGHLAND AVE,85014.0,PARKING LOT
4,201700000000000,11/1/15 12:00 AM,NaN,LARCENY-THEFT,279XX N 23RD LN,85085.0,SINGLE FAMILY HOUSE


In [3]:
crime_data_df.dtypes


INC NUMBER             object
OCCURRED ON            object
OCCURRED TO            object
UCR CRIME CATEGORY     object
100 BLOCK ADDR         object
ZIP                   float64
PREMISE TYPE           object
dtype: object

In [4]:
#Clean up datatypes
crime_data_df['OCCURRED ON'] = pd.to_datetime(crime_data_df['OCCURRED ON'])
crime_data_df['OCCURRED TO'] = pd.to_datetime(crime_data_df['OCCURRED TO'])
crime_data_df['UCR CRIME CATEGORY'] = crime_data_df['UCR CRIME CATEGORY'].apply(str)
crime_data_df['ZIP'] = crime_data_df['ZIP'].apply(str).replace('\.0', '', regex=True)


In [5]:
crime_data_df.dtypes

INC NUMBER                    object
OCCURRED ON           datetime64[ns]
OCCURRED TO           datetime64[ns]
UCR CRIME CATEGORY            object
100 BLOCK ADDR                object
ZIP                           object
PREMISE TYPE                  object
dtype: object

In [31]:
#occurred_on_year = crime_data_df['OCCURRED ON'].dt.year

occurred_on_year = crime_data_df['INC NUMBER'].str[:4].astype(int)

occurred_on_month = crime_data_df['OCCURRED ON'].dt.month
occurred_on_day = crime_data_df['OCCURRED ON'].dt.day

crime_data_df['occurred_on_year'] = occurred_on_year.fillna(0.0).astype(int)
crime_data_df['occurred_on_month'] = occurred_on_month.fillna(0.0).astype(int)
crime_data_df['occurred_on_day'] = occurred_on_day.fillna(0.0).astype(int)

crime_data_df.head()


,INC NUMBER,OCCURRED ON,OCCURRED TO,UCR CRIME CATEGORY,100 BLOCK ADDR,ZIP,PREMISE TYPE,occurred_on_year,occurred_on_month,occurred_on_day
0,201500000000000,2015-11-01,2015-11-01 05:00:00,MOTOR VEHICLE THEFT,102XX W MEDLOCK AVE,85307,SINGLE FAMILY HOUSE,2015,11,1
1,201500000000000,2015-11-01,2015-11-01 11:50:00,MOTOR VEHICLE THEFT,69XX W WOOD ST,85043,SINGLE FAMILY HOUSE,2015,11,1
2,201600000000000,2015-11-01,2016-01-09 00:00:00,MOTOR VEHICLE THEFT,N 43RD AVE & W CACTUS RD,85029,SINGLE FAMILY HOUSE,2016,11,1
3,201500000000000,2015-11-01,2015-11-11 09:30:00,LARCENY-THEFT,14XX E HIGHLAND AVE,85014,PARKING LOT,2015,11,1
4,201700000000000,2015-11-01,NaT,LARCENY-THEFT,279XX N 23RD LN,85085,SINGLE FAMILY HOUSE,2017,11,1


In [7]:
crime_data_df['UCR CRIME CATEGORY'].value_counts()

LARCENY-THEFT                            128858
BURGLARY                                  40756
MOTOR VEHICLE THEFT                       26840
DRUG OFFENSE                              21163
AGGRAVATED ASSAULT                        19323
ROBBERY                                   10761
RAPE                                       3730
ARSON                                      1571
MURDER AND NON-NEGLIGENT MANSLAUGHTER       512
Name: UCR CRIME CATEGORY, dtype: int64

In [8]:
crime_data_df['PREMISE TYPE'].value_counts()

SINGLE FAMILY HOUSE                    44744
APARTMENT                              28981
STREET / ROADWAY / ALLEY / SIDEWALK    25596
PARKING LOT                            25393
RETAIL BUSINESS                        19545
VEHICLE                                13131
CONVENIENCE MARKET / STORE              9218
DEPARTMENT / DISCOUNT STORE             8918
DRIVEWAY                                8849
GROCERY / SUPER MARKET                  6936
OFFICE / COMMERCIAL BUILDING            3953
OTHER                                   3829
HOTEL / MOTEL                           3668
GAS / SERVICE STATION                   3344
SPECIALTY STORE                         3274
RESTAURANT                              3096
CONDO / TOWNHOUSE                       2811
FENCED COMMERCIAL YARD                  2680
PARK / PLAYGROUND                       2537
SINGLE FAMILY HOUSING                   2088
SCHOOL-ELEMENTARY/SECONDARY             1812
DRUG STORE                              1634
FENCED RES

In [32]:
occurred_on_yr = crime_data_df['occurred_on_year']
crime_category = crime_data_df['UCR CRIME CATEGORY']

crime_type_yr = {"occurred_on_yr":occurred_on_yr,
                "crime_category":crime_category}

crime_type_yr_df = pd.DataFrame(crime_type_yr)

crime_type_yr_df.head()

,occurred_on_yr,crime_category
0,2015,MOTOR VEHICLE THEFT
1,2015,MOTOR VEHICLE THEFT
2,2016,MOTOR VEHICLE THEFT
3,2015,LARCENY-THEFT
4,2017,LARCENY-THEFT


In [33]:
crime_type_yr_sum = crime_type_yr_df.groupby(["occurred_on_yr", "crime_category"])['crime_category'].count()
#crime_type_yr_sum.index.name = None
crime_type_yr_sum

occurred_on_yr  crime_category                       
0               LARCENY-THEFT                                1
2015            AGGRAVATED ASSAULT                         575
                ARSON                                       34
                BURGLARY                                  1916
                DRUG OFFENSE                               611
                LARCENY-THEFT                             4999
                MOTOR VEHICLE THEFT                       1134
                MURDER AND NON-NEGLIGENT MANSLAUGHTER       23
                RAPE                                       108
                ROBBERY                                    375
2016            AGGRAVATED ASSAULT                        4108
                ARSON                                      275
                BURGLARY                                 11470
                DRUG OFFENSE                              4660
                LARCENY-THEFT                            31759
 

In [10]:
crime_type_year = crime_data_df.groupby(['occurred_on_year','UCR CRIME CATEGORY']).count()
crime_type_year.head()

,INC NUMBER,OCCURRED ON,OCCURRED TO,UCR CRIME CATEGORY,100 BLOCK ADDR,ZIP,PREMISE TYPE,occurred_on_year,occurred_on_month,occurred_on_day
0,201500000000000,2015-11-01 00:00:00,2015-11-01 05:00:00,MOTOR VEHICLE THEFT,102XX W MEDLOCK AVE,85307,SINGLE FAMILY HOUSE,2015,11,1
1,201500000000000,2015-11-01 00:00:00,2015-11-01 11:50:00,MOTOR VEHICLE THEFT,69XX W WOOD ST,85043,SINGLE FAMILY HOUSE,2015,11,1
2,201600000000000,2015-11-01 00:00:00,2016-01-09 00:00:00,MOTOR VEHICLE THEFT,N 43RD AVE & W CACTUS RD,85029,SINGLE FAMILY HOUSE,2015,11,1
3,201500000000000,2015-11-01 00:00:00,2015-11-11 09:30:00,LARCENY-THEFT,14XX E HIGHLAND AVE,85014,PARKING LOT,2015,11,1
4,201700000000000,2015-11-01 00:00:00,NaT,LARCENY-THEFT,279XX N 23RD LN,85085,SINGLE FAMILY HOUSE,2015,11,1
5,201700000000000,2015-11-01 00:00:00,2016-03-31 00:00:00,RAPE,38XX W CAMELBACK RD,85019,HOSPITAL,2015,11,1
6,201500000000000,2015-11-01 00:00:00,2015-11-01 09:00:00,LARCENY-THEFT,51XX N 15TH ST,85014,APARTMENT,2015,11,1
7,201600000000000,2015-11-01 00:00:00,NaT,RAPE,13XX E ALMERIA RD,85006,SINGLE FAMILY HOUSE,2015,11,1
8,201500000000000,2015-11-01 00:00:00,2015-11-01 15:21:00,DRUG OFFENSE,54XX W INDIAN SCHOOL RD,85031,APARTMENT,2015,11,1
9,201600000000000,2015-11-01 00:00:00,2016-03-22 00:36:00,LARCENY-THEFT,33XX W CAMELBACK RD,85017,PARKING LOT,2015,11,1
